In [8]:
# importing module
from pandas import *
from statistics import mean
import numpy as np
from helper760 import read_inputs

In [9]:
# test case list 
# test_case 0 -> Use CT Data to detect cancer (Keras Neural Network)
# test_case 1 -> Use CT Data to detect cancer (KNN)
# test_case 2 -> Use CT Data + Clinic Data to detect death (Keras Neural Network)
test_case = 2

In [10]:
Clininc_Data,Outcome_Data,CT_Data = read_inputs()
def kFoldCV(k, X, y, model):
    len_x = X.shape[0]
    ind_step = int(len_x  / k)
    ind = np.array([i * ind_step for i in range(k)])
    accuracy = np.zeros(k)
    precision = np.zeros(k)
    recall = np.zeros(k)
    # first k - 1 fold
    for i in range(k - 1):
        X_train_k = np.vstack((X[:ind[i],:],X[ind[i+1]:,:]))
        X_test_k = X[ind[i]:ind[i + 1]]
        y_train_k = np.hstack((y[:ind[i]],y[ind[i+1]:]))
        y_test_k = y[ind[i]:ind[i + 1]]
        _, accuracy[i],precision[i],recall[i] = model.evaluate(X_test_k, y_test_k)       
    # last fold
    X_train_k = X[:ind[k - 1],:]
    y_train_k = y[:ind[k - 1]]
    X_test_k = X[ind[k - 1]:,:]
    y_test_k = y[ind[k - 1]:]
    _, accuracy[k - 1],precision[k - 1],recall[k - 1] = model.evaluate(X_test_k, y_test_k)
    return accuracy,precision,recall

# Test Case 0 - Keras CT-To-Cancer Prediction
This is a test run perform prediction of cancer using CT Data. This test run uses a neural network constructed using Keras

In [4]:
if test_case == 0:
    import numpy as np
    from tensorflow import keras
    from keras.models import Sequential
    from keras.layers import Dense
    from sklearn.preprocessing import StandardScaler 

    X = np.array(CT_Data)
    X = X.T
    X = X.astype(float)
    y = np.array(Outcome_Data[21])
    y = y.astype(int)
    standardScalar = StandardScaler()
    standardScalar.fit(X)
    standardScalar.transform(X)

    # define the keras model
    model = Sequential()
    model.add(Dense(32, input_dim=11, use_bias=True))
    model.add(Dense(64, activation='relu',use_bias=True))
    model.add(Dense(128, activation='relu',use_bias=True))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    # compile the keras model
    model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy','Precision','Recall'])
    # fit the keras model on the dataset
    model.fit(X, y, epochs=1000, batch_size=50,verbose=1)
    # evaluate the keras model
    accuracy,precision,recall = kFoldCV(10, X, y, model)
    print("K fold average accuracy is: {0:.4%},average precision is: {1:.4%},average recall is: {2:.4%}" \
          .format(np.mean(accuracy),np.mean(precision),np.mean(recall)))


    # =========================================================
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    test_y = model.predict(X)
    print(X.shape)
    print(test_y.shape)
    test_y = test_y > 0.5
    right_count = 0
    wrong_count = 0
    cancer_dectected = 0
    for i in range(len(test_y)):
        if test_y[i] == y[i]:
            right_count = right_count + 1
            if test_y[i] == True:
                cancer_dectected = cancer_dectected + 1
        else:
            wrong_count = wrong_count + 1
    print(right_count)
    print(wrong_count)
    print(cancer_dectected)
    print("validation accuracy is: "+str(right_count / (right_count + wrong_count)) + ". Successfully detected cancer: "+str(cancer_dectected))
    print("precision score is: {0:.4%}".format(precision_score(y, test_y)))
    print("recall_score is: {0:.4%}".format(recall_score(y, test_y)))
    print("f1_score: {0:.4%}".format(f1_score(y, test_y)))

Epoch 1/1000
185/185 [==============================] - 1s 932us/step - loss: 5.5085 - accuracy: 0.7551 - precision: 0.1364 - recall: 0.1629
Epoch 2/1000
185/185 [==============================] - 0s 949us/step - loss: 0.6433 - accuracy: 0.7750 - precision: 0.1439 - recall: 0.1447
Epoch 3/1000
185/185 [==============================] - 0s 950us/step - loss: 0.4088 - accuracy: 0.7848 - precision: 0.1473 - recall: 0.1340
Epoch 4/1000
185/185 [==============================] - 0s 938us/step - loss: 0.5531 - accuracy: 0.7781 - precision: 0.1413 - recall: 0.1365
Epoch 5/1000
185/185 [==============================] - 0s 977us/step - loss: 0.6874 - accuracy: 0.7876 - precision: 0.1521 - recall: 0.1356
Epoch 6/1000
185/185 [==============================] - 0s 879us/step - loss: 0.7465 - accuracy: 0.7881 - precision: 0.1554 - recall: 0.1390
Epoch 7/1000
185/185 [==============================] - 0s 879us/step - loss: 0.6098 - accuracy: 0.7730 - precision: 0.1108 - recall: 0.1042
Epoch 8/1000


185/185 [==============================] - 0s 897us/step - loss: 0.1971 - accuracy: 0.8217 - precision: 0.2142 - recall: 0.1348
Epoch 59/1000
185/185 [==============================] - 0s 892us/step - loss: 0.1556 - accuracy: 0.8342 - precision: 0.2411 - recall: 0.1232
Epoch 60/1000
185/185 [==============================] - 0s 890us/step - loss: 0.1709 - accuracy: 0.8380 - precision: 0.2487 - recall: 0.1166
Epoch 61/1000
185/185 [==============================] - 0s 880us/step - loss: 0.2003 - accuracy: 0.8291 - precision: 0.2173 - recall: 0.1166
Epoch 62/1000
185/185 [==============================] - 0s 870us/step - loss: 0.1830 - accuracy: 0.8327 - precision: 0.2315 - recall: 0.1191
Epoch 63/1000
185/185 [==============================] - 0s 926us/step - loss: 0.2146 - accuracy: 0.8207 - precision: 0.1931 - recall: 0.1158
Epoch 64/1000
185/185 [==============================] - 0s 914us/step - loss: 0.1811 - accuracy: 0.8312 - precision: 0.2229 - recall: 0.1158
Epoch 65/1000
185/18

185/185 [==============================] - 0s 896us/step - loss: 0.1118 - accuracy: 0.8603 - precision: 0.3785 - recall: 0.1017
Epoch 116/1000
185/185 [==============================] - 0s 920us/step - loss: 0.1094 - accuracy: 0.8606 - precision: 0.3558 - recall: 0.0786
Epoch 117/1000
185/185 [==============================] - 0s 953us/step - loss: 0.1136 - accuracy: 0.8560 - precision: 0.3422 - recall: 0.1067
Epoch 118/1000
185/185 [==============================] - 0s 922us/step - loss: 0.1089 - accuracy: 0.8615 - precision: 0.3851 - recall: 0.0943
Epoch 119/1000
185/185 [==============================] - 0s 902us/step - loss: 0.1108 - accuracy: 0.8594 - precision: 0.3590 - recall: 0.0926
Epoch 120/1000
185/185 [==============================] - 0s 929us/step - loss: 0.1043 - accuracy: 0.8639 - precision: 0.4050 - recall: 0.0811
Epoch 121/1000
185/185 [==============================] - 0s 916us/step - loss: 0.1090 - accuracy: 0.8582 - precision: 0.3377 - recall: 0.0852
Epoch 122/1000

185/185 [==============================] - 0s 926us/step - loss: 0.0916 - accuracy: 0.8743 - precision: 0.6202 - recall: 0.1067
Epoch 173/1000
185/185 [==============================] - 0s 930us/step - loss: 0.0978 - accuracy: 0.8656 - precision: 0.4460 - recall: 0.1059
Epoch 174/1000
185/185 [==============================] - 0s 884us/step - loss: 0.0928 - accuracy: 0.8724 - precision: 0.5678 - recall: 0.1108
Epoch 175/1000
185/185 [==============================] - 0s 880us/step - loss: 0.0912 - accuracy: 0.8723 - precision: 0.5671 - recall: 0.1084
Epoch 176/1000
185/185 [==============================] - 0s 872us/step - loss: 0.0929 - accuracy: 0.8708 - precision: 0.5341 - recall: 0.1100
Epoch 177/1000
185/185 [==============================] - 0s 872us/step - loss: 0.0938 - accuracy: 0.8687 - precision: 0.4958 - recall: 0.0976
Epoch 178/1000
185/185 [==============================] - 0s 878us/step - loss: 0.0932 - accuracy: 0.8720 - precision: 0.5648 - recall: 0.1009
Epoch 179/1000

185/185 [==============================] - 0s 884us/step - loss: 0.0837 - accuracy: 0.8790 - precision: 0.6576 - recall: 0.1605
Epoch 230/1000
185/185 [==============================] - 0s 869us/step - loss: 0.0822 - accuracy: 0.8819 - precision: 0.7344 - recall: 0.1555
Epoch 231/1000
185/185 [==============================] - 0s 882us/step - loss: 0.0828 - accuracy: 0.8840 - precision: 0.7509 - recall: 0.1720
Epoch 232/1000
185/185 [==============================] - 0s 876us/step - loss: 0.0826 - accuracy: 0.8795 - precision: 0.6828 - recall: 0.1514
Epoch 233/1000
185/185 [==============================] - 0s 882us/step - loss: 0.0834 - accuracy: 0.8794 - precision: 0.6559 - recall: 0.1687
Epoch 234/1000
185/185 [==============================] - 0s 899us/step - loss: 0.0873 - accuracy: 0.8764 - precision: 0.6012 - recall: 0.1696
Epoch 235/1000
185/185 [==============================] - 0s 934us/step - loss: 0.0809 - accuracy: 0.8832 - precision: 0.7558 - recall: 0.1613
Epoch 236/1000

...

185/185 [==============================] - 0s 938us/step - loss: 0.0260 - accuracy: 0.9642 - precision: 0.9435 - recall: 0.7734
Epoch 743/1000
185/185 [==============================] - 0s 908us/step - loss: 0.0254 - accuracy: 0.9629 - precision: 0.9322 - recall: 0.7734
Epoch 744/1000
185/185 [==============================] - 0s 869us/step - loss: 0.0247 - accuracy: 0.9652 - precision: 0.9476 - recall: 0.7775
Epoch 745/1000
185/185 [==============================] - 0s 869us/step - loss: 0.0250 - accuracy: 0.9638 - precision: 0.9442 - recall: 0.7692
Epoch 746/1000
185/185 [==============================] - 0s 870us/step - loss: 0.0255 - accuracy: 0.9624 - precision: 0.9327 - recall: 0.7684
Epoch 747/1000
185/185 [==============================] - 0s 872us/step - loss: 0.0282 - accuracy: 0.9587 - precision: 0.9182 - recall: 0.7519
Epoch 748/1000
185/185 [==============================] - 0s 869us/step - loss: 0.0277 - accuracy: 0.9587 - precision: 0.9083 - recall: 0.7618
Epoch 749/1000

185/185 [==============================] - 0s 883us/step - loss: 0.0257 - accuracy: 0.9631 - precision: 0.9239 - recall: 0.7833
Epoch 800/1000
185/185 [==============================] - 0s 862us/step - loss: 0.0224 - accuracy: 0.9679 - precision: 0.9394 - recall: 0.8073
Epoch 801/1000
185/185 [==============================] - 0s 921us/step - loss: 0.0220 - accuracy: 0.9705 - precision: 0.9553 - recall: 0.8131
Epoch 802/1000
185/185 [==============================] - 0s 872us/step - loss: 0.0232 - accuracy: 0.9676 - precision: 0.9452 - recall: 0.7990
Epoch 803/1000
185/185 [==============================] - 0s 931us/step - loss: 0.0219 - accuracy: 0.9690 - precision: 0.9538 - recall: 0.8023
Epoch 804/1000
185/185 [==============================] - 0s 929us/step - loss: 0.0248 - accuracy: 0.9649 - precision: 0.9259 - recall: 0.7957
Epoch 805/1000
185/185 [==============================] - 0s 867us/step - loss: 0.0362 - accuracy: 0.9506 - precision: 0.8617 - recall: 0.7419
Epoch 806/1000

185/185 [==============================] - 0s 873us/step - loss: 0.0205 - accuracy: 0.9702 - precision: 0.9422 - recall: 0.8230
Epoch 857/1000
185/185 [==============================] - 0s 921us/step - loss: 0.0191 - accuracy: 0.9727 - precision: 0.9570 - recall: 0.8288
Epoch 858/1000
185/185 [==============================] - 0s 922us/step - loss: 0.0189 - accuracy: 0.9743 - precision: 0.9551 - recall: 0.8437
Epoch 859/1000
185/185 [==============================] - 0s 971us/step - loss: 0.0198 - accuracy: 0.9737 - precision: 0.9557 - recall: 0.8379
Epoch 860/1000
185/185 [==============================] - 0s 907us/step - loss: 0.0226 - accuracy: 0.9658 - precision: 0.9241 - recall: 0.8056
Epoch 861/1000
185/185 [==============================] - 0s 927us/step - loss: 0.0219 - accuracy: 0.9688 - precision: 0.9407 - recall: 0.8131
Epoch 862/1000
185/185 [==============================] - 0s 869us/step - loss: 0.0191 - accuracy: 0.9743 - precision: 0.9611 - recall: 0.8379
Epoch 863/1000

185/185 [==============================] - 0s 913us/step - loss: 0.0166 - accuracy: 0.9760 - precision: 0.9600 - recall: 0.8528
Epoch 914/1000
185/185 [==============================] - 0s 916us/step - loss: 0.0173 - accuracy: 0.9760 - precision: 0.9660 - recall: 0.8470
Epoch 915/1000
185/185 [==============================] - 0s 864us/step - loss: 0.0189 - accuracy: 0.9734 - precision: 0.9505 - recall: 0.8412
Epoch 916/1000
185/185 [==============================] - 0s 928us/step - loss: 0.0189 - accuracy: 0.9724 - precision: 0.9492 - recall: 0.8337
Epoch 917/1000
185/185 [==============================] - 0s 937us/step - loss: 0.0194 - accuracy: 0.9714 - precision: 0.9371 - recall: 0.8379
Epoch 918/1000
185/185 [==============================] - 0s 886us/step - loss: 0.0239 - accuracy: 0.9661 - precision: 0.9203 - recall: 0.8114
Epoch 919/1000
185/185 [==============================] - 0s 931us/step - loss: 0.0255 - accuracy: 0.9655 - precision: 0.9223 - recall: 0.8048
Epoch 920/1000

185/185 [==============================] - 0s 924us/step - loss: 0.0231 - accuracy: 0.9682 - precision: 0.9241 - recall: 0.8255
Epoch 971/1000
185/185 [==============================] - 0s 886us/step - loss: 0.0468 - accuracy: 0.9392 - precision: 0.7951 - recall: 0.7221
Epoch 972/1000
185/185 [==============================] - 0s 916us/step - loss: 0.0377 - accuracy: 0.9531 - precision: 0.8593 - recall: 0.7676
Epoch 973/1000
185/185 [==============================] - 0s 862us/step - loss: 0.0177 - accuracy: 0.9751 - precision: 0.9520 - recall: 0.8528
Epoch 974/1000
185/185 [==============================] - 0s 921us/step - loss: 0.0156 - accuracy: 0.9785 - precision: 0.9642 - recall: 0.8685
Epoch 975/1000
185/185 [==============================] - 0s 930us/step - loss: 0.0149 - accuracy: 0.9786 - precision: 0.9642 - recall: 0.8693
Epoch 976/1000
185/185 [==============================] - 0s 923us/step - loss: 0.0156 - accuracy: 0.9769 - precision: 0.9586 - recall: 0.8610
Epoch 977/1000

validation accuracy is: 0.9664968014745744. Successfully detected cancer: 918
precision score is: 98.0769%
recall_score is: 75.9305%
f1_score: 85.5944%

# Test Run - KNN CT-To-Cancer Prediction
In this test run we tried to use CT_Data to predict cancer, the algorithm we used here is K-nearest-Neighbours

In [5]:
if test_case == 1:
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler

    param_grid = [
        {
            'weights': ['uniform'], 
            'n_neighbors': [i for i in range(1, 20)]
        },
        {
            'weights': ['distance'],
            'n_neighbors': [i for i in range(1, 20)], 
            'p': [i for i in range(1, 6)]
        }
    ]



    X = np.array(CT_Data)
    X = X.T
    X = X.astype(float)
    standardScalar = StandardScaler()
    standardScalar.fit(X)
    standardScalar.transform(X)
    y = np.array(Outcome_Data[21])
    y = y.astype(int)
    print(X.shape)
    print(y.shape)
    knn_clf = KNeighborsClassifier()
    grid_search = GridSearchCV(knn_clf, param_grid , cv = 5, scoring = 'average_precision')
    grid_search.fit(X, y)
    print(grid_search.best_estimator_)
    print(grid_search.best_score_)


    # ============================================
    def predict_Knn(test_vec, train_data, train_label, k):
        res_list = []
        dis_list = []
        train_data = train_data.astype(float)
        test_vec = test_vec.astype(float)
        for i in range(train_data.shape[1]):
            diff = (test_vec.reshape((train_data.shape[0],1))) - train_data[:,i]
            dis_list.append(np.linalg.norm(diff))

            
        # Smallest K elements indices
        # using sorted() + lambda + list slicing
        res = sorted(range(len(dis_list)), key = lambda sub: dis_list[sub])[:k] 

        for i in range(len(res)):
            res_list.append(train_label[res[i]])

        count_0 = 0
        count_1 = 0

        for i in range(len(res_list)):
            if res_list[i] == 0:
                count_0 = count_0 + 1
            else:
                count_1 = count_1 + 1

        #if count_1 > count_0:
        #    return 1
        #else:
        #    return 0        

        # as long as one nearest neighbour has cancer,
        # than we predict cancer
        if count_1 > 0:
            return 1
        else:
            return 0
    

# Test Case 2 - Keras CT+Clinic_Data-To-Death Prediction
This is a test run perform prediction of death using (CT Data + Clinic Data). This test run uses a neural network constructed using Keras

In [11]:
if test_case == 2:
    import numpy as np
    from tensorflow import keras
    from keras.models import Sequential
    from keras.layers import Dense
    from sklearn.preprocessing import StandardScaler 

    X1 = np.array(CT_Data)
    X1 = X1.T
    X1 = X1.astype(float)
    print(X1.shape)

    X2 = np.array(Clininc_Data)
    X2 = X2.T
    X2 = X2.astype(float)
    print(X2.shape)

    X = np.hstack((X1,X2))
    print(X.shape)

    y = np.array(Outcome_Data[0])
    y = y.astype(int)
    standardScalar = StandardScaler()
    standardScalar.fit(X)
    standardScalar.transform(X)

    # define the keras model
    model = Sequential()
    model.add(Dense(32, input_dim=25, use_bias=True))
    model.add(Dense(64, activation='relu',use_bias=True))
    model.add(Dense(128, activation='relu',use_bias=True))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    # compile the keras model
    model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy','Precision','Recall'])
    # fit the keras model on the dataset
    model.fit(X, y, epochs=1000, batch_size=50,verbose=1)
    # evaluate the keras model
    _, accuracy,precision,recall = model.evaluate(X, y)
    print('Accuracy: %.2f' % (accuracy*100))
    print('precision: %.2f' % (precision*100))
    print('recall rate: %.2f' % (recall*100))
    accuracy,precision,recall = kFoldCV(10, X, y, model)
    print("K fold average accuracy is: {0:.4%},average precision is: {1:.4%},average recall is: {2:.4%}" \
          .format(np.mean(accuracy),np.mean(precision),np.mean(recall)))


    # =========================================================
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    test_y = model.predict(X)
    print(X.shape)
    print(test_y.shape)
    test_y = test_y > 0.5
    right_count = 0
    wrong_count = 0
    death_dectected = 0
    for i in range(len(test_y)):
        if test_y[i] == y[i]:
            right_count = right_count + 1
            if test_y[i] == True:
                death_dectected = death_dectected + 1
        else:
            wrong_count = wrong_count + 1
    print(right_count)
    print(wrong_count)
    print(death_dectected)
    print("validation accuracy is: "+str(right_count / (right_count + wrong_count)) + ". Successfully detected death: "+str(death_dectected))
    print("precision score is: {0:.4%}".format(precision_score(y, test_y)))
    print("recall_score is: {0:.4%}".format(recall_score(y, test_y)))
    print("f1_score: {0:.4%}".format(f1_score(y, test_y)))

(9223, 11)
(9223, 14)
(9223, 25)
Epoch 1/1000
185/185 [==============================] - 1s 864us/step - loss: 6.3212 - accuracy: 0.8995 - precision: 0.1337 - recall: 0.1257  
Epoch 2/1000
185/185 [==============================] - 0s 899us/step - loss: 2.4247 - accuracy: 0.9036 - precision: 0.1718 - recall: 0.1621
Epoch 3/1000
185/185 [==============================] - 0s 887us/step - loss: 1.8524 - accuracy: 0.9063 - precision: 0.2045 - recall: 0.1985
Epoch 4/1000
185/185 [==============================] - 0s 898us/step - loss: 1.6890 - accuracy: 0.9046 - precision: 0.1871 - recall: 0.1803
Epoch 5/1000
185/185 [==============================] - 0s 897us/step - loss: 1.7400 - accuracy: 0.9055 - precision: 0.2048 - recall: 0.2040
Epoch 6/1000
185/185 [==============================] - 0s 910us/step - loss: 1.7198 - accuracy: 0.9079 - precision: 0.2180 - recall: 0.2113
Epoch 7/1000
185/185 [==============================] - 0s 909us/step - loss: 1.4095 - accuracy: 0.9123 - precision: 0.

185/185 [==============================] - 0s 893us/step - loss: 0.6394 - accuracy: 0.9283 - precision: 0.3982 - recall: 0.3989
Epoch 59/1000
185/185 [==============================] - 0s 924us/step - loss: 0.8156 - accuracy: 0.9245 - precision: 0.3651 - recall: 0.3625
Epoch 60/1000
185/185 [==============================] - 0s 915us/step - loss: 0.6292 - accuracy: 0.9238 - precision: 0.3640 - recall: 0.3752
Epoch 61/1000
185/185 [==============================] - 0s 895us/step - loss: 0.7282 - accuracy: 0.9246 - precision: 0.3643 - recall: 0.3570
Epoch 62/1000
185/185 [==============================] - 0s 891us/step - loss: 0.5108 - accuracy: 0.9275 - precision: 0.3958 - recall: 0.4153
Epoch 63/1000
185/185 [==============================] - 0s 940us/step - loss: 0.6209 - accuracy: 0.9284 - precision: 0.3982 - recall: 0.3953
Epoch 64/1000
185/185 [==============================] - 0s 907us/step - loss: 0.5588 - accuracy: 0.9309 - precision: 0.4197 - recall: 0.4189
Epoch 65/1000
185/18

185/185 [==============================] - 0s 881us/step - loss: 0.3348 - accuracy: 0.9347 - precision: 0.4529 - recall: 0.4645
Epoch 116/1000
185/185 [==============================] - 0s 881us/step - loss: 0.2414 - accuracy: 0.9364 - precision: 0.4662 - recall: 0.4772
Epoch 117/1000
185/185 [==============================] - 0s 880us/step - loss: 0.2481 - accuracy: 0.9353 - precision: 0.4583 - recall: 0.4809
Epoch 118/1000
185/185 [==============================] - 0s 881us/step - loss: 0.4237 - accuracy: 0.9325 - precision: 0.4315 - recall: 0.4244
Epoch 119/1000
185/185 [==============================] - 0s 885us/step - loss: 0.3947 - accuracy: 0.9339 - precision: 0.4450 - recall: 0.4499
Epoch 120/1000
185/185 [==============================] - 0s 878us/step - loss: 0.3058 - accuracy: 0.9365 - precision: 0.4678 - recall: 0.4900
Epoch 121/1000
185/185 [==============================] - 0s 886us/step - loss: 0.4611 - accuracy: 0.9308 - precision: 0.4183 - recall: 0.4153
Epoch 122/1000

185/185 [==============================] - 0s 898us/step - loss: 0.1645 - accuracy: 0.9415 - precision: 0.5079 - recall: 0.5246
Epoch 173/1000
185/185 [==============================] - 0s 918us/step - loss: 0.1864 - accuracy: 0.9398 - precision: 0.4947 - recall: 0.5082
Epoch 174/1000
185/185 [==============================] - 0s 900us/step - loss: 0.2207 - accuracy: 0.9361 - precision: 0.4654 - recall: 0.4900
Epoch 175/1000
185/185 [==============================] - 0s 897us/step - loss: 0.2133 - accuracy: 0.9395 - precision: 0.4919 - recall: 0.4973
Epoch 176/1000
185/185 [==============================] - 0s 896us/step - loss: 0.1851 - accuracy: 0.9432 - precision: 0.5222 - recall: 0.5355
Epoch 177/1000
185/185 [==============================] - 0s 893us/step - loss: 0.2009 - accuracy: 0.9396 - precision: 0.4928 - recall: 0.4991
Epoch 178/1000
185/185 [==============================] - 0s 882us/step - loss: 0.1690 - accuracy: 0.9405 - precision: 0.5000 - recall: 0.5118
Epoch 179/1000

185/185 [==============================] - 0s 893us/step - loss: 0.0894 - accuracy: 0.9508 - precision: 0.5859 - recall: 0.5902
Epoch 230/1000
185/185 [==============================] - 0s 893us/step - loss: 0.1059 - accuracy: 0.9451 - precision: 0.5390 - recall: 0.5410
Epoch 231/1000
185/185 [==============================] - 0s 892us/step - loss: 0.1054 - accuracy: 0.9474 - precision: 0.5567 - recall: 0.5719
Epoch 232/1000
185/185 [==============================] - 0s 903us/step - loss: 0.0885 - accuracy: 0.9465 - precision: 0.5496 - recall: 0.5647
Epoch 233/1000
185/185 [==============================] - 0s 949us/step - loss: 0.1021 - accuracy: 0.9460 - precision: 0.5447 - recall: 0.5665
Epoch 234/1000
185/185 [==============================] - 0s 937us/step - loss: 0.0983 - accuracy: 0.9470 - precision: 0.5545 - recall: 0.5556
Epoch 235/1000
185/185 [==============================] - 0s 887us/step - loss: 0.1585 - accuracy: 0.9364 - precision: 0.4662 - recall: 0.4772
Epoch 236/1000

...

185/185 [==============================] - 0s 882us/step - loss: 0.0102 - accuracy: 0.9885 - precision: 0.9187 - recall: 0.8852
Epoch 743/1000
185/185 [==============================] - 0s 878us/step - loss: 0.0080 - accuracy: 0.9898 - precision: 0.9350 - recall: 0.8907
Epoch 744/1000
185/185 [==============================] - 0s 880us/step - loss: 0.0108 - accuracy: 0.9867 - precision: 0.9019 - recall: 0.8707
Epoch 745/1000
185/185 [==============================] - 0s 881us/step - loss: 0.0108 - accuracy: 0.9881 - precision: 0.9262 - recall: 0.8689
Epoch 746/1000
185/185 [==============================] - 0s 877us/step - loss: 0.0087 - accuracy: 0.9899 - precision: 0.9351 - recall: 0.8925
Epoch 747/1000
185/185 [==============================] - 0s 886us/step - loss: 0.0105 - accuracy: 0.9871 - precision: 0.9266 - recall: 0.8506
Epoch 748/1000
185/185 [==============================] - 0s 884us/step - loss: 0.0147 - accuracy: 0.9845 - precision: 0.8904 - recall: 0.8434
Epoch 749/1000

185/185 [==============================] - 0s 943us/step - loss: 0.0027 - accuracy: 0.9970 - precision: 0.9851 - recall: 0.9636
Epoch 800/1000
185/185 [==============================] - 0s 950us/step - loss: 0.0105 - accuracy: 0.9880 - precision: 0.9132 - recall: 0.8816
Epoch 801/1000
185/185 [==============================] - 0s 927us/step - loss: 0.0260 - accuracy: 0.9752 - precision: 0.8042 - recall: 0.7705
Epoch 802/1000
185/185 [==============================] - 0s 951us/step - loss: 0.0233 - accuracy: 0.9791 - precision: 0.8436 - recall: 0.7960
Epoch 803/1000
185/185 [==============================] - 0s 911us/step - loss: 0.0312 - accuracy: 0.9717 - precision: 0.7769 - recall: 0.7359
Epoch 804/1000
185/185 [==============================] - 0s 892us/step - loss: 0.0079 - accuracy: 0.9900 - precision: 0.9352 - recall: 0.8944
Epoch 805/1000
185/185 [==============================] - 0s 898us/step - loss: 0.0044 - accuracy: 0.9951 - precision: 0.9719 - recall: 0.9454
Epoch 806/1000

185/185 [==============================] - 0s 899us/step - loss: 0.0376 - accuracy: 0.9717 - precision: 0.7697 - recall: 0.7486
Epoch 857/1000
185/185 [==============================] - 0s 880us/step - loss: 0.0135 - accuracy: 0.9856 - precision: 0.8939 - recall: 0.8597
Epoch 858/1000
185/185 [==============================] - 0s 882us/step - loss: 0.0064 - accuracy: 0.9943 - precision: 0.9662 - recall: 0.9362
Epoch 859/1000
185/185 [==============================] - 0s 893us/step - loss: 0.0036 - accuracy: 0.9949 - precision: 0.9718 - recall: 0.9417
Epoch 860/1000
185/185 [==============================] - 0s 887us/step - loss: 0.0063 - accuracy: 0.9931 - precision: 0.9516 - recall: 0.9308   
Epoch 861/1000
185/185 [==============================] - 0s 883us/step - loss: 0.0077 - accuracy: 0.9922 - precision: 0.9526 - recall: 0.9144
Epoch 862/1000
185/185 [==============================] - 0s 897us/step - loss: 0.0035 - accuracy: 0.9956 - precision: 0.9774 - recall: 0.9472
Epoch 863/1

185/185 [==============================] - 0s 896us/step - loss: 0.0086 - accuracy: 0.9894 - precision: 0.9247 - recall: 0.8944
Epoch 914/1000
185/185 [==============================] - 0s 883us/step - loss: 0.0031 - accuracy: 0.9958 - precision: 0.9793 - recall: 0.9490
Epoch 915/1000
185/185 [==============================] - 0s 880us/step - loss: 0.0027 - accuracy: 0.9973 - precision: 0.9834 - recall: 0.9709
Epoch 916/1000
185/185 [==============================] - 0s 892us/step - loss: 0.0025 - accuracy: 0.9970 - precision: 0.9833 - recall: 0.9654
Epoch 917/1000
185/185 [==============================] - 0s 879us/step - loss: 0.0022 - accuracy: 0.9969 - precision: 0.9815 - recall: 0.9654
Epoch 918/1000
185/185 [==============================] - 0s 888us/step - loss: 0.0029 - accuracy: 0.9965 - precision: 0.9814 - recall: 0.9599
Epoch 919/1000
185/185 [==============================] - 0s 894us/step - loss: 0.0157 - accuracy: 0.9856 - precision: 0.8866 - recall: 0.8689
Epoch 920/1000

185/185 [==============================] - 0s 898us/step - loss: 0.0086 - accuracy: 0.9886 - precision: 0.9126 - recall: 0.8944
Epoch 971/1000
185/185 [==============================] - 0s 892us/step - loss: 0.0053 - accuracy: 0.9958 - precision: 0.9757 - recall: 0.9526
Epoch 972/1000
185/185 [==============================] - 0s 896us/step - loss: 0.0061 - accuracy: 0.9943 - precision: 0.9697 - recall: 0.9326
Epoch 973/1000
185/185 [==============================] - 0s 882us/step - loss: 0.0077 - accuracy: 0.9944 - precision: 0.9645 - recall: 0.9399
Epoch 974/1000
185/185 [==============================] - 0s 908us/step - loss: 0.0079 - accuracy: 0.9928 - precision: 0.9481 - recall: 0.9308
Epoch 975/1000
185/185 [==============================] - 0s 891us/step - loss: 0.0150 - accuracy: 0.9842 - precision: 0.8780 - recall: 0.8525
Epoch 976/1000
185/185 [==============================] - 0s 899us/step - loss: 0.0083 - accuracy: 0.9898 - precision: 0.9205 - recall: 0.9071
Epoch 977/1000

validation accuracy is: 0.9998915754093028. Successfully detected death: 548
precision score is: 100.0000%
recall_score is: 99.8179%
f1_score: 99.9088%

# Test Case 3 - Keras CT-To-Death Prediction
This is a test run perform prediction of death using (CT Data Only). This test run uses a neural network constructed using Keras

In [7]:
if test_case == 3:
    import numpy as np
    from tensorflow import keras
    from keras.models import Sequential
    from keras.layers import Dense
    from sklearn.preprocessing import StandardScaler 

    X = np.array(CT_Data)
    X = X.T
    X = X.astype(float)


    y = np.array(Outcome_Data[0])
    y = y.astype(int)
    standardScalar = StandardScaler()
    standardScalar.fit(X)
    standardScalar.transform(X)

    # define the keras model
    model = Sequential()
    model.add(Dense(32, input_dim=25, use_bias=True))
    model.add(Dense(64, activation='relu',use_bias=True))
    model.add(Dense(128, activation='relu',use_bias=True))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    # compile the keras model
    model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy','Precision','Recall'])
    # fit the keras model on the dataset
    model.fit(X, y, epochs=1000, batch_size=50,verbose=1)
    # evaluate the keras model
    _, accuracy,precision,recall = model.evaluate(X, y)
    print('Accuracy: %.2f' % (accuracy*100))
    print('precision: %.2f' % (precision*100))
    print('recall rate: %.2f' % (recall*100))
    


    # =========================================================
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    test_y = model.predict(X)
    print(X.shape)
    print(test_y.shape)
    test_y = test_y > 0.5
    right_count = 0
    wrong_count = 0
    death_dectected = 0
    for i in range(len(test_y)):
        if test_y[i] == y[i]:
            right_count = right_count + 1
            if test_y[i] == True:
                death_dectected = death_dectected + 1
        else:
            wrong_count = wrong_count + 1
    print(right_count)
    print(wrong_count)
    print(death_dectected)
    print("validation accuracy is: "+str(right_count / (right_count + wrong_count)) + ". Successfully detected death: "+str(death_dectected))
    print("precision score is: {0:.4%}".format(precision_score(y, test_y)))
    print("recall_score is: {0:.4%}".format(recall_score(y, test_y)))
    print("f1_score: {0:.4%}".format(f1_score(y, test_y)))